In [1]:
import dask.dataframe as dd
import numpy as np
from dask.diagnostics import ProgressBar 
from scipy.signal import find_peaks
import hvplot.dask
import hvplot.pandas
# import graphviz

In [2]:
pbar = ProgressBar()
pbar.register()
n_partition = 200

In [48]:
data = dd.read_parquet('D:\\joystick\\test.parquet')
#data = data.repartition(n_partition)
#data = data.reset_index(drop=True)
data = data.set_index(data.index, sorted=True)
print(data.npartitions)
#data.dask

ValueError: New division must be list or tuple

In [47]:
data.divisions

(None, None)

In [24]:
print(f'Estimated total memory usage: {((data.memory_usage().sum().compute())/(1024**3)):.3f} GByte')

[########################################] | 100% Completed | 110.68 ms
Estimated total memory usage: 0.003 GByte


In [39]:
print(data)

Dask DataFrame Structure:
              port_1_p(bar) port_2_p(bar) port_3_p(bar) port_4_p(bar) support_p(bar) tank_p(bar) temp(c) time(hour)
npartitions=1                                                                                                      
                    float32       float32       float32       float32        float32     float32   uint8    float32
                        ...           ...           ...           ...            ...         ...     ...        ...
Dask Name: read_parquet, 1 expression
Expr=ReadParquetFSSpec(2b60437)


In [40]:
data.head()

[########################################] | 100% Completed | 108.85 ms


,port_1_p(bar),port_2_p(bar),port_3_p(bar),port_4_p(bar),support_p(bar),tank_p(bar),temp(c),time(hour)
0,0.55,12.29,12.42,0.53,12.37,0.53,48,0.000000
1,0.55,12.32,12.45,0.55,12.37,0.54,48,0.000003
2,0.54,12.31,12.43,0.54,12.35,0.54,48,0.000006
3,0.58,12.32,12.43,0.54,12.38,0.55,48,0.000008
4,0.55,12.30,12.43,0.54,12.36,0.55,48,0.000011


In [ ]:
data.tail()

In [ ]:
hvplot.extension('bokeh')

data.hvplot.line(x='time(hour)', y='port_1_p(bar)', rasterize=True)

In [41]:
peaks, _ = find_peaks(data['port_1_p(bar)'], height=10, distance=100*60*1)
index = dd.from_array(np.arange(len(peaks), dtype=np.uint32), columns='index')
print(len(peaks))

[########################################] | 100% Completed | 114.34 ms
16


In [42]:
p1_peak = data.loc[peaks]

KeyError: 'Cannot index with list against unknown division. Try setting divisions using ``ddf.set_index``'

In [6]:
p1_peak_df = p1_peak.compute()

[########################################] | 100% Completed | 70.61 s


In [11]:
p1_peak_df.drop(columns=['level_0'], inplace=True)

In [20]:
p1_peak_df.describe()

,index,port_1_p(bar),port_2_p(bar),port_3_p(bar),port_4_p(bar),support_p(bar),tank_p(bar),temp(c),time(hour)
count,6.259800e+04,62598.000000,62598.000000,62598.000000,62598.000000,62598.000000,62598.000000,62598.000000,62598.000000
mean,2.558537e+08,21.145182,1.751784,1.748514,1.864535,40.797443,1.593927,48.002636,710.704651
std,1.453157e+08,2.038701,0.739705,0.157018,0.690368,0.565820,0.140285,0.369850,403.654846
min,3.116000e+03,11.300000,0.650000,0.760000,0.720000,35.619999,0.670000,18.000000,0.008656
25%,1.323804e+08,19.690001,1.450000,1.640000,1.600000,40.680000,1.500000,48.000000,367.723366
50%,2.571213e+08,22.080000,1.590000,1.770000,1.760000,40.910000,1.610000,48.000000,714.225891
75%,3.813317e+08,22.809999,1.840000,1.870000,1.870000,41.099998,1.700000,48.000000,1059.254700
max,5.058236e+08,23.780001,15.140000,2.960000,11.440000,48.660000,2.580000,63.000000,1405.065552


In [21]:
p1_peak_df.reset_index(inplace=True, drop=True)

In [24]:
p1_peak_df

,index,port_1_p(bar),port_2_p(bar),port_3_p(bar),port_4_p(bar),support_p(bar),tank_p(bar),temp(c),time(hour)
0,3116,23.110001,1.50,1.60,10.48,38.740002,1.54,48,0.008656
1,14311,23.190001,0.93,0.99,0.99,39.509998,0.94,48,0.039753
2,23274,23.129999,0.94,1.13,1.09,38.980000,1.05,48,0.064650
3,29574,22.889999,0.88,0.95,0.98,39.000000,0.91,47,0.082150
4,35921,23.160000,0.81,0.95,0.89,38.959999,0.88,47,0.099781
...,...,...,...,...,...,...,...,...,...
62593,505788388,15.990000,1.58,1.74,2.30,41.299999,1.59,48,1404.967773
62594,505797428,16.100000,1.48,1.61,2.88,40.169998,1.51,48,1404.992798
62595,505805819,16.059999,1.54,1.67,1.63,40.139999,1.54,48,1405.016113
62596,505815355,16.219999,1.42,1.55,1.55,41.029999,1.45,48,1405.042603


In [25]:
hvplot.extension('bokeh')

p1_peak_df.hvplot.scatter(x='time(hour)', y='port_1_p(bar)', title='peak pressure', xlabel='time(hour)', ylabel='pressure(bar)', ylim=(0, 25), rasterize=False, width=1920, height=1080, grid=True,
                       fontsize={'title':'20pt', 'labels': '20pt', 'xticks': '15pt', 'yticks': '15pt'}, colorbar=False)

:Scatter   [time(hour)]   (port_1_p(bar))